In [264]:
import pandas as pd
import numpy as np
import nltk
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [265]:
df_analyze = pd.read_csv('data/payments_main.tsv',index_col=False,sep='\t', names = ['id','date','payment','text'])

In [266]:
df_analyze.head()

,id,date,payment,text
0,1,07.11.2024,40500.00,За тур.поездку по договору №001 от 27.01.2023г
1,2,07.11.2024,"32600,00",За оказание услуг по договору №53Б-02746 от 23...
2,3,07.11.2024,4710-00,Оплата штрафа
3,4,07.11.2024,30900-00,Лечение по договору №Д-00359/24 от 08.03.2025
4,5,07.11.2024,13200.00,Оплата основного долга за период с 16.12.2024г...


In [267]:
df_analyze = df_analyze.drop(columns = ['id','date'])

In [268]:
df_analyze.head()

,payment,text
0,40500.00,За тур.поездку по договору №001 от 27.01.2023г
1,"32600,00",За оказание услуг по договору №53Б-02746 от 23...
2,4710-00,Оплата штрафа
3,30900-00,Лечение по договору №Д-00359/24 от 08.03.2025
4,13200.00,Оплата основного долга за период с 16.12.2024г...


#### Step-1: Remove non-cyrillic symbols

In [269]:
def remove_non_cyrillic(text):
 cyrillic_pattern = r"[^а-яА-ЯёЁ]+"
 cleaned_text = re.sub(cyrillic_pattern, " ", text)
 return cleaned_text

In [270]:
df_analyze['text'] = df_analyze['text'].apply(remove_non_cyrillic)

In [271]:
df_analyze.head()

,payment,text
0,40500.00,За тур поездку по договору от г
1,"32600,00",За оказание услуг по договору Б от
2,4710-00,Оплата штрафа
3,30900-00,Лечение по договору Д от
4,13200.00,Оплата основного долга за период с г по марта ...


#### Step-1.5 Remove joined words

def split_joined_words(text):
    splited_text = text.split(' ')
    tokens = []
    splited = False
    for token in splited_text:
        splited = False
        for i in range(1, len(token)):
            token = token.strip()
            if not token[i-1].isupper() and token[i].isupper():
                token1,token2 = token[0:i], token[i:]
                splited = True
                print(token1,token2)
                tokens.append(f' {token1} ')
                tokens.append(f' {token2} ')
        if splited is False:
            tokens.append(f' {token} ')
    return ''.join(tokens)

In [272]:
def split_joined_words(text):
    
    tokens = []
    
    def split_token(split_points, text):           
        for i in range(1,len(split_points)):
            token = text[split_points[i-1]:split_points[i]]
            tokens.append(f' {token} ')
    
    splited_text = text.split(' ')
    for token in splited_text:
        split_points = [0]
        for i in range(1, len(token)):
            token = token.strip()
            if not token[i-1].isupper() and token[i].isupper():
                split_points.append(i)
        split_points.append(len(token))
        split_token(split_points,token)
                
    return ''.join(tokens)

In [273]:
df_analyze['text'] = df_analyze['text'].apply(split_joined_words)

#### Step-2: Apply stop words and lemmatize

In [274]:
stopwords_ru = stopwords.words("russian")
stopwords_ru.extend( {"по", "от", "до", "с", "в", "на", "за", "к", "о", "об", "у", "со", "из", "при", "под", "про", "через", "над", "без"})
morph = MorphAnalyzer()

In [275]:
def remove_stop_words(text):
    text_splited = text.split(' ')
    filtered_tokens = []
    for token in text_splited:
        if token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            if len(token) < 2: # Отсев единичных букв
                continue
            filtered_tokens.append(f' {token} ')
    # if len(filtered_tokens) <= 2:
    #    return None # ( ' ' ?)
    return ''.join(filtered_tokens)

In [276]:
df_analyze['text'] = df_analyze['text'].apply(remove_stop_words)

In [277]:
df_analyze.head()

,payment,text
0,40500.00,за тур поездка договор
1,"32600,00",за оказание услуга договор
2,4710-00,оплата штраф
3,30900-00,лечение договор далее
4,13200.00,оплата основный долг период март договор...


#### Step-3: TF-IDF

In [278]:
texts = df_analyze['text']

In [279]:
texts_raw = texts.values
unique_tokens = []
for text in texts_raw:
    tokens = text.split(' ')
    for token in tokens:
        if token not in unique_tokens:
            unique_tokens.append(token)

count_unique_tokens = len(unique_tokens)
print(f' Уникальных токенов: {count_unique_tokens}')

 Уникальных токенов: 1786


In [280]:
tfidf = TfidfVectorizer(max_features=count_unique_tokens) 

In [281]:
tfidf_matrix = tfidf.fit_transform(texts).toarray()

#### Step-4: PCA

In [285]:
pca_transform = PCA(n_components = 744)

In [286]:
tfidf_matrix_compressed = pca_transform.fit_transform(tfidf_matrix)

In [287]:
print(f'Компонент до сжатия: {tfidf_matrix.shape[1]}, компонент после сжатия: {tfidf_matrix_compressed.shape[1]}')

Компонент до сжатия: 1785, компонент после сжатия: 744


In [291]:
explained_variance = np.sum(pca_transform.explained_variance_ratio_)
print(f'Величина объясненной дисперсии: {explained_variance }')

Величина объясненной дисперсии: 0.9744697971216831
